In [ ]:
%config IPCompleter.greedy=True
%matplotlib inline

# Import the dependencies.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("../data/weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2020-08-28 14:15:04,-46.19,168.86,45.00,72,97,11.01
1,1,Shimoda,JP,2020-08-28 14:15:04,34.67,138.95,73.00,95,0,2.55
2,2,Cape Town,ZA,2020-08-28 14:14:47,-33.93,18.42,55.99,76,75,12.75
3,3,Kaitangata,NZ,2020-08-28 14:15:05,-46.28,169.85,42.01,80,96,11.97
4,4,Kemin,KG,2020-08-28 14:15:05,42.78,75.69,61.65,45,0,3.29


In [4]:
# Recall that you use the dtypes method to get the data types of a DataFrame.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# 1. Assign the locations to an array of latitude and longitude pairs.
# locations = [latitude, longitude]
# 2. Assign the weights variable to some values.
# temperatures = # an array of equal length of the locations array.
# 3. Assign the figure variable to the gmaps.figure() attribute.
# fig = gmaps.figure()
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
# heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)

# 5. Add the heatmap layer.
# fig.add_layer(heatmap_layer)
# 6. Call the figure to plot the data.
# fig

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [9]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [10]:
# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
windspeed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=windspeed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  70
What is the maximum temperature you would like for your trip?  101


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Shimoda,JP,2020-08-28 14:15:04,34.67,138.95,73.00,95,0,2.55
5,5,Vanimo,PG,2020-08-28 14:15:05,-2.67,141.30,80.46,83,100,8.32
6,6,Urubicha,BO,2020-08-28 14:15:05,-15.38,-62.95,95.02,26,32,4.05
7,7,San Cristobal,VE,2020-08-28 14:15:05,7.77,-72.22,82.40,61,20,4.70
8,8,Bambous Virieux,MU,2020-08-28 14:13:20,-20.34,57.76,73.40,83,75,11.41
11,11,Kapaa,US,2020-08-28 14:11:04,22.08,-159.32,72.00,59,6,10.67
12,12,Hambantota,LK,2020-08-28 14:15:06,6.12,81.12,86.00,66,90,11.41
13,13,Kahului,US,2020-08-28 14:15:06,20.89,-156.47,73.99,69,1,3.36
14,14,Butaritari,KI,2020-08-28 14:15:06,3.07,172.79,83.12,69,99,10.63
16,16,Surt,LY,2020-08-28 14:15:07,31.21,16.59,84.00,56,0,12.44


In [13]:
preferred_cities_df.count()

City_ID       310
City          310
Country       307
Date          310
Lat           310
Lng           310
Max Temp      310
Humidity      310
Cloudiness    310
Wind Speed    310
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Shimoda,JP,73.00,34.67,138.95,
5,Vanimo,PG,80.46,-2.67,141.30,
6,Urubicha,BO,95.02,-15.38,-62.95,
7,San Cristobal,VE,82.40,7.77,-72.22,
8,Bambous Virieux,MU,73.40,-20.34,57.76,
11,Kapaa,US,72.00,22.08,-159.32,
12,Hambantota,LK,86.00,6.12,81.12,
13,Kahului,US,73.99,20.89,-156.47,
14,Butaritari,KI,83.12,3.07,172.79,
16,Surt,LY,84.00,31.21,16.59,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")
hotel_df.head()        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Shimoda,JP,73.00,34.67,138.95,Shimoda Tokyu Hotel
5,Vanimo,PG,80.46,-2.67,141.30,Vahmoneh Lodge Vanimo
6,Urubicha,BO,95.02,-15.38,-62.95,
7,San Cristobal,VE,82.40,7.77,-72.22,Pirineos
8,Bambous Virieux,MU,73.40,-20.34,57.76,Casa Tia Villa


In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [18]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))